# SOME BACKGROUND

~ STEP 1 :- First download the latest Python Veriosn 
            https://www.python.org/downloads/

Please Install the downloaded file on your computer using custom Installation settings. 

~ STEP 2 :- After the installation is done , Open  , In MAC - Terminal or WIN - Terminal , and type "Python3 --version"
 for MAC or "Python --version" for WIN and press Enter 

 If , the Terminal , returns with the verion of Python like "Python 3.11.1" , Installation has been succesful , and then we can proceed further , if not , please check if STEP 1 has been accomplished

~STEP 3 :- After we are done  with the STEP 2 , I am using the Jupytr Lab for the tutorials. , type in terminal "pip install jupyter lab" , this will install the latest verison of Jupyter Lab on your system , automatically you will be redirected to the browser 

~ STEP 4 :- To check , if the installation of Jupyter lab has been successful , type "jupyter lab --version" on Terminal. If , the Terminal , returns with the verion of Jupyter Lab like "4.1.2" , Installation has been succesful , and then we can proceed further , if not , please check if STEP 3 has been accomplished

~ STEP 5 :- A library is a collection of code that makes everyday tasks more efficient. For my tutorial i would recommend Installing pandas , numpy - For data analysis , seaborn , matplotlib - For Visualisation.  

Also , i would be Installing a library called "peakutils" , this woulbe be useful for recogisation of Peaks from the Spectra. https://peakutils.readthedocs.io/en/latest/


~ STEP 6 :- For installing the library , go to terminal , and type "pip install pandas" and after installing . Similarly , do this for numpy , seaborn , matplotlin and PeakUtils

~ STEP 7 :- Now , to open the Jupyter Lab seession in the browser , go to terminal and type "jupyter lab" , you will be automatically redirected to the browser in a jupyter lab session. This is where you will start programming . 

NOTE :- To exit the session , press "Ctrl + C" on the termianal , this will end the running session on your system

~ STEP 8 :- In jupyter lab , your lines of codes or chunk of lines of codes  are called as "Cell" , If the "Cell are of type Markdown - Used for Comments / Information " and If the "Cells are of type Code - Used for Programming" 

You can get more in fo here -  https://www.datacamp.com/tutorial/installing-jupyter-notebook


# Importing the Useful librararies. 

Line [1] is completely optional , This is used to Hide the the Warnings in the Output cell. It's recommended to skip the line[1] , and proceed with the code from line[2].

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show

from bokeh.io import output_notebook
output_notebook()


Loading BokehJS ...

# LOADING THE SPECTRA FILE WHICH WE HAVE

In [3]:
file_path = 'New_Spectra/CR210LA/CR210LA_1.csv'

Sample_Spectra = pd.read_csv(file_path)
Sample_Spectra = Sample_Spectra.rename({' intensity':'Intensities'},axis=1)


Sample_Spectra_Plot = figure(title = 'Raw Sample Spectra' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Plot.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="green" )
Sample_Spectra_Plot.width = 1200
Sample_Spectra_Plot.height = 600

print(max(Sample_Spectra.Intensities))

print (Sample_Spectra)
show (Sample_Spectra_Plot)



29348.15746306448
       wavelength  Intensities
0      180.000000          0.0
1      180.033333          0.0
2      180.066667          0.0
3      180.100000          0.0
4      180.133333          0.0
...           ...          ...
23426  960.866667          0.0
23427  960.900000          0.0
23428  960.933333          0.0
23429  960.966667          0.0
23430  961.000000          0.0

[23431 rows x 2 columns]


##### Now , as we see in the figure avove , the spectra have lots of lines , which are very specific to the element present in the Sample . 

Each element has an specific , wavelength associated with it.

Now , i have visited the NIST LIBS database , and download the spectra for all the element in the Periodic Table. The File are updated with that of NIST database as of 17 April 2024.

# CHOOSING THE ELEMENT OF INTEREST

In [4]:

element = 'Mo' # ENTER YOUR ELEMENT OF CHOICE
Reference_Spectra = pd.read_csv(r"NIST/" + element +".csv" , encoding = 'windows - 1252' , usecols=(0,1))
print(Reference_Spectra)

Reference_Spectra_Plot = figure(title = 'NIST Reference Library ', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Reference_Spectra_Plot.line(Reference_Spectra.Wavelength,Reference_Spectra.Spectra , line_width = 2, color ="blue" )
Reference_Spectra_Plot.width = 1200
Reference_Spectra_Plot.height = 600

show(Reference_Spectra_Plot)

      Wavelength       Spectra
0         253.27  9.260000e-10
1         253.31  6.100000e-09
2         253.35  3.820000e-08
3         253.39  2.280000e-07
4         253.44  1.990000e-06
...          ...           ...
9839      770.09  4.070000e-09
9840      770.13  2.220000e-09
9841      770.17  1.200000e-09
9842      770.22  5.550000e-10
9843      770.26  2.970000e-10

[9844 rows x 2 columns]


# APPROACH FOR PEAK DETECTION IN THE NIST ELEMENT SPECTRA

Now, uptil now , we have selected the Material Sample ,which we want to Interact with and the data is stored in variable "Sample_Spectra"
Also , we have choosen the element , which we want to detect in this 'Sample_Spectra' , and the spectral information for that element is stored in variable "Reference_Spectra" 

Now , from this 'Reference_Spectra' , Let's try to get all the Peaks this Spectra has

I have used a library called as Peak_Utils 

https://peakutils.readthedocs.io/en/latest/reference.html

The detail of the function is given below. 

---------------------------------------------------------------------------------------------------------------------------

NOTE :- Now if we keep the minimum_distance_1 = 0 and threshold_1 = 0 , we get all the peaks in the spectra , Tuning of these parameter are most important as per your signal.

Setting the value in parameter 'minimum_distance_1' , will allows to skip the peaks , which are very close to each other. 
Setting the value in parameter 'threshold_1', will allows to skip the peaks with low intensity and also seperate the Peaks , with could be the noise signal. 

So , its important to choose the value's as per your convenience 

----------------------------------------------------------------------------------------------------------------------------

In [5]:
minimum_distance_1 = 0 
threshold_1 = 0 #looking for a value above 0.01% , so that Peak with very low Intensities can be ignored 

Reference_Peak = indexes(Reference_Spectra.Spectra , min_dist=minimum_distance_1 , thres=threshold_1) 

Selected_Reference_Spectra = Reference_Spectra.iloc[Reference_Peak] #from the indices , lets try to find the Wavelength and Spectra. 

Reference_Spectra_Peaks = figure(title = 'Peaks in NIST Spectra ', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Reference_Spectra_Peaks.line(Reference_Spectra.Wavelength,Reference_Spectra.Spectra , line_width = 2, color ="blue" )
Reference_Spectra_Peaks.circle(Selected_Reference_Spectra.Wavelength , Selected_Reference_Spectra.Spectra , color = 'red' , size = 8)

Reference_Spectra_Peaks.width = 1200
Reference_Spectra_Peaks.height = 600

print(Selected_Reference_Spectra)
show(Reference_Spectra_Peaks)



      Wavelength     Spectra
36        254.80  10800000.0
81        256.71  29100000.0
158       261.26  67800000.0
199       263.00  65000000.0
223       264.02  74300000.0
...          ...         ...
9014      675.41    803000.0
9206      683.87   1340000.0
9318      688.63    649000.0
9521      744.75    667000.0
9735      765.67   1410000.0

[187 rows x 2 columns]


# Wavelengths for Detected Peak(red circles above)

In [6]:
Reference_Spectra_List = []

for iteam in Selected_Reference_Spectra.Wavelength:
    iteam = float(iteam)
    if iteam > 187.0:        #Only those iteam will be selected , whose wavelength would be greater than 188 nm ,  as our Handheld LIBS range starts from 188nm
        Reference_Spectra_List.append(iteam)
    
print(Reference_Spectra_List) #These are the wavelengths of Peaks , which we are interested in the NIST Spectra
Length_Reference_Spectra_List = (len(Reference_Spectra_List))   #Used for Calculating Percentage Detection

[254.8, 256.71, 261.26, 263.0, 264.02, 264.96, 265.72, 266.49, 267.98, 270.15, 270.61, 272.53, 273.33, 274.44, 275.16, 276.18, 278.78, 280.05, 282.64, 286.47, 287.66, 288.64, 290.55, 291.49, 293.06, 293.66, 294.59, 295.95, 297.78, 298.89, 300.08, 301.39, 302.5, 304.2, 304.67, 305.6, 306.41, 307.43, 308.54, 309.47, 310.11, 311.22, 313.26, 314.7, 315.81, 317.04, 318.49, 319.42, 320.74, 322.14, 323.25, 324.82, 325.63, 326.39, 327.03, 328.9, 330.65, 332.52, 334.51, 335.87, 338.43, 340.42, 342.0, 343.61, 344.68, 346.04, 346.89, 348.37, 350.5, 352.12, 354.16, 355.81, 357.05, 358.2, 360.07, 361.13, 362.45, 363.85, 364.83, 366.02, 368.1, 369.46, 370.87, 371.93, 372.82, 374.78, 375.84, 376.94, 377.97, 379.84, 383.15, 384.81, 386.43, 388.68, 390.29, 394.29, 395.57, 397.4, 399.18, 401.01, 402.11, 406.24, 407.0, 408.41, 410.7, 412.02, 413.17, 415.72, 418.78, 423.24, 424.56, 425.29, 427.71, 429.11, 432.6, 434.09, 438.17, 441.14, 442.38, 443.48, 444.76, 445.69, 447.44, 449.14, 451.69, 453.69, 459.81

# 0.0333 is the sprectral resolution of the Handheld LIBS Device

Now , for the two wavelengths from two different data source , to exactly match with each other , is going to be difficult because of the spectral resolution of wavelength - for these two data source are very different from each other.

Now we know that the Spectral Resolution of Handheld LIBS Device is 0.0333. 

Now , we have a list of all wavelengths , detected for Peaks in Reference Spectra , Now for each element lets try to create a min and max of each element in the list. 'Reference_Spectra_Min_Max' is list consisting of a sublist , the sublist have a min and max value or a range of element in the list.


Eg : - A peak of 254.8 in NIST Spectra , can be found at and between [254.7668 ~ 254.8332] in the Handheld LIBS Spectra , because the resolution of wavelength in NIST Spectra and Handheld LIBS Spectra is different. 


In [7]:
Reference_Spectra_Min_Max = []
Reference_Spectra_Min_Max = [[num - 0.0332, num + 0.0332] for num in Reference_Spectra_List]     
    
print(Reference_Spectra_Min_Max)

[[254.76680000000002, 254.8332], [256.67679999999996, 256.7432], [261.22679999999997, 261.2932], [262.9668, 263.0332], [263.98679999999996, 264.0532], [264.92679999999996, 264.9932], [265.6868, 265.75320000000005], [266.4568, 266.52320000000003], [267.9468, 268.01320000000004], [270.11679999999996, 270.1832], [270.5768, 270.64320000000004], [272.49679999999995, 272.5632], [273.29679999999996, 273.3632], [274.4068, 274.4732], [275.1268, 275.19320000000005], [276.1468, 276.21320000000003], [278.74679999999995, 278.8132], [280.0168, 280.08320000000003], [282.60679999999996, 282.6732], [286.4368, 286.50320000000005], [287.6268, 287.69320000000005], [288.60679999999996, 288.6732], [290.5168, 290.58320000000003], [291.4568, 291.52320000000003], [293.0268, 293.0932], [293.6268, 293.69320000000005], [294.55679999999995, 294.6232], [295.91679999999997, 295.9832], [297.74679999999995, 297.8132], [298.85679999999996, 298.9232], [300.04679999999996, 300.1132], [301.35679999999996, 301.4232], [302.

# Pre-Processing the Spectra for Handheld LIBS to improve the accuracy of Peak Detection 

As we see in our raw spectra , there are lot of small peaks , which really aren't emission peaks , but are the noise in the signal. To tackle that Smoothing the Spectra is usually done. 

I am sing the Savitsky Golay , for smoothing my spectra. 

# Noise Estimation
As we see in the Handheld LIBS Spectra , It's difficult to find or detect , what is the noise and what is the signal.

But we have an information , that the range of Spectrometer in hLIBS starts from 190nm , So we can say that the signal detected uptil 190nm could be classified as noise.

In [8]:
noise_list = [] 
noise_start = []

noise_list = Sample_Spectra.loc[Sample_Spectra['wavelength'] < 190.0 , 'Intensities']

RMS_intensity_noise = max(noise_list) #np.sqrt(np.mean((noise_list)**2)) #This is the max Intensity of the noise signal
print(RMS_intensity_noise)  



268.6938677162817


In [9]:
#METHOD  - Scipy

Threshold = 4 #(Noise Intensity * Threshold) Value for signal filtering
height_2 = None
Peak_Prominenace = None # #Uses S/N ratio , as comapred to other peaks
minimum_distance_3 = None   #minimum distance between two peaks

Sample_Peak , _ = fp(Sample_Spectra.Intensities,height=height_2,prominence=Peak_Prominenace,distance=minimum_distance_3)
Selected_Sample_Spectra = Sample_Spectra.iloc[Sample_Peak] 

Sample_Spectra_Peaks = figure(title = 'Peaks in hLIBS Spectra', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Peaks.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="green" )
Sample_Spectra_Peaks.circle(Selected_Sample_Spectra.wavelength , Selected_Sample_Spectra.Intensities , color = 'red' , size = 8)

Sample_Spectra_Peaks.width = 1200
Sample_Spectra_Peaks.height = 600

print(Selected_Sample_Spectra)
show(Sample_Spectra_Peaks)

       wavelength  Intensities
242    188.066667   169.003109
251    188.366667    61.505109
259    188.633333    61.201582
266    188.866667   268.693868
273    189.100000    80.804517
...           ...          ...
23025  947.500000    46.641869
23037  947.900000    44.665809
23057  948.566667    39.180918
23069  948.966667    38.258725
23080  949.333333    36.289069

[1527 rows x 2 columns]


# Wavelengths for Detected Peak(red circles above)

In [10]:
Sample_Spectra_List = []

for iteam in Selected_Sample_Spectra.wavelength:
    iteam = float(iteam)
    Sample_Spectra_List.append(iteam)
    
print(Sample_Spectra_List)


[188.0666666666662, 188.3666666666662, 188.63333333333284, 188.86666666666616, 189.09999999999948, 189.5333333333328, 189.8333333333328, 190.1333333333328, 190.46666666666607, 190.6999999999994, 191.06666666666604, 191.466666666666, 191.83333333333263, 192.23333333333264, 192.59999999999928, 193.06666666666592, 193.29999999999924, 193.49999999999923, 193.63333333333256, 193.86666666666588, 194.0333333333325, 194.3999999999992, 194.6333333333325, 194.8333333333325, 194.99999999999915, 195.23333333333247, 195.53333333333245, 195.7999999999991, 195.96666666666576, 196.16666666666572, 196.3333333333324, 196.5333333333324, 196.89999999999904, 197.1333333333324, 197.499999999999, 197.799999999999, 197.96666666666565, 198.2333333333323, 198.63333333333227, 199.03333333333225, 199.43333333333223, 199.89999999999887, 200.06666666666555, 200.4666666666655, 200.7666666666655, 201.0999999999988, 201.66666666666544, 202.59999999999872, 204.09999999999863, 204.86666666666525, 205.1333333333319, 205.

# So far so now , i have two list with me 

1) Sample_Spectra_List = Containg wavelngths of all Peaks from data of Handheld LIBS device
2) Reference_Spectra_Min_Max = Containing a range of wavelength , for all the peak detected in the Reference Sample

   

In [11]:
print(Sample_Spectra_List)

[188.0666666666662, 188.3666666666662, 188.63333333333284, 188.86666666666616, 189.09999999999948, 189.5333333333328, 189.8333333333328, 190.1333333333328, 190.46666666666607, 190.6999999999994, 191.06666666666604, 191.466666666666, 191.83333333333263, 192.23333333333264, 192.59999999999928, 193.06666666666592, 193.29999999999924, 193.49999999999923, 193.63333333333256, 193.86666666666588, 194.0333333333325, 194.3999999999992, 194.6333333333325, 194.8333333333325, 194.99999999999915, 195.23333333333247, 195.53333333333245, 195.7999999999991, 195.96666666666576, 196.16666666666572, 196.3333333333324, 196.5333333333324, 196.89999999999904, 197.1333333333324, 197.499999999999, 197.799999999999, 197.96666666666565, 198.2333333333323, 198.63333333333227, 199.03333333333225, 199.43333333333223, 199.89999999999887, 200.06666666666555, 200.4666666666655, 200.7666666666655, 201.0999999999988, 201.66666666666544, 202.59999999999872, 204.09999999999863, 204.86666666666525, 205.1333333333319, 205.

In [12]:
print(Reference_Spectra_Min_Max)

[[254.76680000000002, 254.8332], [256.67679999999996, 256.7432], [261.22679999999997, 261.2932], [262.9668, 263.0332], [263.98679999999996, 264.0532], [264.92679999999996, 264.9932], [265.6868, 265.75320000000005], [266.4568, 266.52320000000003], [267.9468, 268.01320000000004], [270.11679999999996, 270.1832], [270.5768, 270.64320000000004], [272.49679999999995, 272.5632], [273.29679999999996, 273.3632], [274.4068, 274.4732], [275.1268, 275.19320000000005], [276.1468, 276.21320000000003], [278.74679999999995, 278.8132], [280.0168, 280.08320000000003], [282.60679999999996, 282.6732], [286.4368, 286.50320000000005], [287.6268, 287.69320000000005], [288.60679999999996, 288.6732], [290.5168, 290.58320000000003], [291.4568, 291.52320000000003], [293.0268, 293.0932], [293.6268, 293.69320000000005], [294.55679999999995, 294.6232], [295.91679999999997, 295.9832], [297.74679999999995, 297.8132], [298.85679999999996, 298.9232], [300.04679999999996, 300.1132], [301.35679999999996, 301.4232], [302.

# Finally , My  idea is see , if each iteam in (Reference_Spectra_Min_Max) , matches with each element in (Sample_Spectra_List) , and then caluclate the exact matching Percentage 


In [13]:
plot_spectra_wavelength_list=[]
plot_reference_wavelength_list = []
count = 0
for cell in Reference_Spectra_Min_Max: 
    for iteam in Sample_Spectra_List:
        if iteam >= cell[0] and iteam <= cell[1]:
            count = count + 1 #for counting
            plot_spectra_wavelength_list.append(iteam)
            plot_reference_wavelength_list.append(cell[0] + 0.0333)

plot_spectra_intensities_list=[]
for iteam in plot_spectra_wavelength_list:
    Inte = Selected_Sample_Spectra.loc[Selected_Sample_Spectra['wavelength'] == iteam, 'Intensities'].iloc[0]
    plot_spectra_intensities_list.append(Inte)
#print(plot_spectra_intensities_list)

#print(plot_spectra_wavelength_list)

print('The total peaks detected were ' + str(count) + '\n' )

print('The following peaks , for Reference Sample was detected  :- '  )
print(plot_reference_wavelength_list )

print('\n')
print('The Sample Peaks detected from the sample were:-')
spectra_df = pd.DataFrame({'wavelength': plot_spectra_wavelength_list, 'intensities': plot_spectra_intensities_list})
print(spectra_df)  



# MAX_Intensity = max(spectra_df.intensities)

# # ###########################################################################

# matching_row = Sample_Spectra[Sample_Spectra['Intensities'] == MAX_Intensity]
# print(matching_row)

The total peaks detected were 41

The following peaks , for Reference Sample was detected  :- 
[264.96009999999995, 270.15009999999995, 270.6101, 273.33009999999996, 276.1801, 278.78009999999995, 280.0501, 287.6601, 290.5501, 293.6601, 295.95009999999996, 300.08009999999996, 301.39009999999996, 317.0401, 322.14009999999996, 325.63009999999997, 334.51009999999997, 340.4201, 346.89009999999996, 350.5001, 352.1201, 354.1601, 357.0501, 363.8501, 364.83009999999996, 370.8701, 379.84009999999995, 390.2901, 413.1701, 432.6001, 444.76009999999997, 501.4701, 510.95009999999996, 514.5301000000001, 519.2001000000001, 532.4301, 550.6601, 570.5601, 572.9401000000001, 582.5101000000001, 613.0401]


The Sample Peaks detected from the sample were:-
    wavelength   intensities
0   264.933333    521.969884
1   270.133333    434.937239
2   270.633333   1117.445771
3   273.333333   1088.838899
4   276.166667   1760.145179
5   278.766667   1166.051170
6   280.066667   6755.069655
7   287.666667    335.540

# RESULTS

In [14]:
percentage = (count / Length_Reference_Spectra_List) * 100
percentage

print (' Ananlysis of ' + element + ' was done , and the ' + str(count) +  ' number of spectral lines from the material sample ' + ' matched with the ' + str(Length_Reference_Spectra_List) + ' choosen Reference element of '+ element + ' therefore , we can say' ,  str(percentage) + '%' + ' matched as per the parameters set by the user')

 Ananlysis of Mo was done , and the 41 number of spectral lines from the material sample  matched with the 187 choosen Reference element of Mo therefore , we can say 21.92513368983957% matched as per the parameters set by the user


Find the Correponding Intensities , for the wavelengths available in "plot_spectra_wavelength_list" and also for the wavelengths available in "plot_refernce_intensities_list"

# Generate a Interactive Plot

In [15]:
p = figure(title = 'Peak Detection' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensities')
p.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="red" )
p.circle(spectra_df.wavelength, spectra_df.intensities , color = 'green' , size = 5)

p.width = 1200
p.height = 600

show(p)

